[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1h4zgINa0NkyBqYWGT7nBwjnaCprCRbLJ?usp=sharing)

In [1]:
!pip3 install -q crewai crewai_tools

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.7/197.7 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.0/463.0 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
from crewai import Agent, Task, Crew

In [4]:
# Add your utilities or helper functions to this file.

import os
from dotenv import load_dotenv, find_dotenv

# these expect to find a .env file at the directory above the lesson.                                                                                                                     # the format for that file is (without the comment)                                                                                                                                       #API_KEYNAME=AStringThatIsTheLongAPIKeyFromSomeService
def load_env():
    _ = load_dotenv(find_dotenv())

def get_openai_api_key():
    load_env()
    openai_api_key = os.getenv("OPENAI_API_KEY")
    return openai_api_key

def get_serper_api_key():
    load_env()
    openai_api_key = os.getenv("SERPER_API_KEY")
    return openai_api_key


# break line every 80 characters if line is longer than 80 characters
# don't break in the middle of a word
def pretty_print_result(result):
  parsed_result = []
  for line in result.split('\n'):
      if len(line) > 80:
          words = line.split(' ')
          new_line = ''
          for word in words:
              if len(new_line) + len(word) + 1 > 80:
                  parsed_result.append(new_line)
                  new_line = word
              else:
                  if new_line == '':
                      new_line = word
                  else:
                      new_line += ' ' + word
          parsed_result.append(new_line)
      else:
          parsed_result.append(line)
  return "\n".join(parsed_result)


In [5]:
import os

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'
os.environ["OPENAI_API_KEY"] = openai_api_key

# Create Customer Support Agent

In [6]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work as a PRICAI 2024 Organizer (https://pricai.org/2024/index.php) and "
        " are now working on providing "
		"support to {customer}, a super important customer or audience "
        " for your conference."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)

# Define Tools

In [8]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

In [9]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://pricai.org/2024/index.php/calls/call-for-tutorials2"
)

# Define Task

In [10]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

# Combine

In [11]:
crew = Crew(
  agents=[support_agent],
  tasks=[inquiry_resolution],
  verbose=True,
  memory=True
)

In [12]:
inputs = {
    "customer": "Kalbe Digital Lab",
    "person": "Adhi Setiawan",
    "inquiry": "I need know about PRICAI 2024 conference, specifically"
               "can you explain detail information regarding call for tutorials?"
}
result = crew.kickoff(inputs=inputs)

# Agent: Senior Support Representative
## Task: Kalbe Digital Lab just reached out with a super important ask:
I need know about PRICAI 2024 conference, specificallycan you explain detail information regarding call for tutorials?

Adhi Setiawan from Kalbe Digital Lab is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


# Agent: Senior Support Representative
## Thought: Thought: To provide detailed information regarding the call for tutorials for the PRICAI 2024 conference, I need to access the specific section on the PRICAI 2024 website related to the call for tutorials.
## Using tool: Read website content
## Tool Input: 
"{}"
## Tool Output: 
Call for Tutorials
HOMEORGANIZATIONOrganizing Committee PC MembersCALLSCall for PapersCall For WorkshopsCall for TutorialsCall For Posters Call for SponsorshipCall for Domestic Volunteers (JP)Call for Student 

In [15]:
def extract_markdown_from_crew(content):
    """
    Extracts raw markdown text from CrewOutput object.

    Args:
        content: CrewOutput object or string

    Returns:
        str: The raw markdown text
    """
    try:
        # If it's a CrewOutput object, extract the raw content
        if hasattr(content, 'raw'):
            return content.raw
        # If it's already a string, return it directly
        return str(content)

    except Exception as e:
        raise Exception(f"Error extracting markdown: {str(e)}")


# Extract the markdown
try:
  markdown_text = extract_markdown_from_crew(result)
  print(markdown_text)  # This will print the raw markdown text
except Exception as e:
  print(f"Error: {e}")

Hi Adhi Setiawan from Kalbe Digital Lab,

Thank you for reaching out with your inquiry regarding the PRICAI 2024 conference's call for tutorials. It's my pleasure to provide you with detailed information.

The PRICAI-2024 organizers have announced a call for proposals for the Tutorial Program at the 21st Pacific Rim International Conference on Artificial Intelligence, to be held in Kyoto, Japan, from November 18-20, 2024. The intention behind these tutorials is to facilitate a platform where researchers and practitioners can delve into the exciting advancements within the AI principles and practices. This initiative is crucial for fostering cross-fertilization, collaboration, and the dynamism of the AI field, promoting the continuing education of the PRICAI community members.

Here are the key objectives and details for the tutorials:

1. **Objectives**:
   - Introduce researchers to major research topics within the PRICAI community.
   - Provide training in proven methodologies and pr